# Quickstart: Live Data via yfinance

This notebook is a light, end-to-end example using the automated yfinance
connector. It is intentionally shorter than the CSV demos.

If you want a full method tour, see:
- `examples/aapl_volcurve_probcurve_demo.ipynb`
- `examples/aapl_volsurface_probsurface_demo.ipynb`

## Setup

Note: This notebook requires an active internet connection.

In [ ]:
from datetime import date

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from oipd import VolCurve, VolSurface, MarketInputs, sources

## Fetch a single-expiry chain

In [ ]:
ticker = "AAPL"

# List available expiries
expiries = sources.list_expiry_dates(ticker)
expiries[:5]

In [ ]:
# Use the first expiry for a single-expiry VolCurve
single_expiry = expiries[0]
chain, snapshot = sources.fetch_chain(ticker, expiries=single_expiry)

market = MarketInputs(
    valuation_date=snapshot.date,
    risk_free_rate=0.04,
    underlying_price=snapshot.underlying_price,
)

curve = VolCurve().fit(chain, market)

{
    "expiry": single_expiry,
    "forward_price": curve.forward_price,
    "atm_vol": curve.atm_vol,
}

In [ ]:
curve.plot(include_observed=True)
plt.show()

## ProbCurve quick check

In [ ]:
prob_curve = curve.implied_distribution()
prob_curve.plot(kind="pdf")
plt.show()

## Fetch multiple expiries and fit a VolSurface

In [ ]:
surface_expiries = expiries[:3]
chain_surface, snapshot_surface = sources.fetch_chain(ticker, expiries=surface_expiries)

surface_market = MarketInputs(
    valuation_date=snapshot_surface.date,
    risk_free_rate=0.04,
    underlying_price=snapshot_surface.underlying_price,
)

surface = VolSurface().fit(chain_surface, surface_market)

surface.expiries

In [ ]:
surface.plot()
plt.show()

In [ ]:
surface.plot_term_structure()
plt.show()

## ProbSurface fan chart

In [ ]:
prob_surface = surface.implied_distribution()
prob_surface.plot_fan(lower_percentile=10.0, upper_percentile=90.0)
plt.show()

## Next steps

If you want a deeper tour of all methods (Greeks, quantiles, slicing, and
interpolation), use the AAPL CSV notebooks in `examples/`.